<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Keras_WithDeeplizard/blob/master/6_Build_A_Validation_Set_With_TensorFlow's_Keras_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build A Validation Set With TensorFlow's Keras API
在本集中，我们将演示如何在训练期间使用TensorFlow的Keras API即时创建验证集。

我们将继续使用在上一集中构建和训练的相同模型，但首先，让我们讨论一下验证集到底是什么。

## 1.什么是验证集

回想一下，我们以前建立了一个训练集，在这个训练集上训练我们的模型。随着我们的模型被训练的每一个epoch，模型将继续学习这个训练集中数据的特征和特征。

我们的希望是，以后我们可以利用这个模型，将它应用到新的数据中，并且让这个模型仅仅基于它从训练集中学到的东西，就能准确地预测它以前从未见过的数据。

现在，让我们讨论一下添加验证集的作用。

在训练开始之前，我们可以选择删除一部分训练集并将其放入验证集中。 然后，在训练期间，模型将仅在训练集上进行训练，并且它将通过评估验证集中的数据进行验证。

本质上，该模型是在训练集中学习数据的特征，从该数据中获取所学信息，然后在验证集中进行预测。 在每个时期，我们不仅会看到训练集的损失和准确性结果，还会看到验证集的损失和准确性结果。

这样一来，我们就可以了解该模型对未经训练的数据进行概括的程度，因为回想一下，验证数据不应该作为训练数据的一部分。

这也有助于我们查看模型是否过拟合。 当模型仅学习训练数据的细节并且无法很好地归纳未训练的数据时，就会发生过拟合。

如果您想进一步了解过度拟合问题，请查看“深度学习基础知识”系列中的过度拟合一集。 请注意，您还可以在该系列中看到训练集和验证集的更深入细分。

现在让我们讨论一下如何创建验证集。

## 2. 创建验证集
有两种方法可以创建用于tf.keras.Sequential模型的验证集。

### 2.1 手工创建验证集
第一种方法是创建一个数据结构来保存验证集，并按照与训练集相同的方式将数据直接放入该结构中。

此数据结构应为Numpy数组或张量的元组valid_set =（x_val，y_val），其中x_val是包含验证样本的numpy数组或张量，而y_val是包含验证标签的numpy数组或张量。

当我们调用model.fit（）时，除了训练集之外，我们还将传入验证集。 我们通过指定validation_data参数来传递验证集。



```
model.fit(
      x=scaled_train_samples
    , y=train_labels
    , validation_data=valid_set
    , batch_size=10
    , epochs=30
    , verbose=2
)
```
当模型进行训练时，它将继续只在训练集上进行训练，但除此之外，它还将评估验证集。


### 2.2 使用Keras创建验证集
创建验证集的另一种方法可以节省步骤！

如果尚未创建指定的验证集，则在调用model.fit（）时，可以为validation_split参数设置一个值。 它期望分数在0到1之间。假设我们将此参数设置为0.1。



```
model.fit(
      x=scaled_train_samples
    , y=train_labels
    , validation_split=0.1
    , batch_size=10
    , epochs=30
    , verbose=2
)
```
指定此参数后，Keras将分割训练数据的一部分（在此示例中为10％），以用作验证数据。 模型将分开训练数据的这一部分，不对其进行训练，并且将在每个时期结束时评估此数据的损失和任何模型度量。

**注意：**默认情况下，fit（）函数会在每个时期之前对数据进行随机排序。 但是，当指定validation_split参数时，将在混洗之前从x和y数据中的最后一个样本中选择验证数据

因此，在我们以这种方式使用validation_split创建验证数据的情况下，我们需要确保我们的数据已提前打乱，就像我们先前在较早的情节中所做的那样。

## 3.Interpret Validation Metrics
现在,无论哪一种方法我们使用创建验证数据,当我们调用model.fit(),那么除了损失和准确性被显示为每个时代为我们上次看到的,我们现在也看到val损失和val acc追踪损失和准确性的验证集

In [ ]:
# 准备数据
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

train_labels = []
train_samples = []

# 生成数据
for i in range(50):
    # 大约5%的年轻人确实经历过副作用
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(1)

    # 大约5%的老年人没有经历过副作用
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(0)

for i in range(1000):
    # 大约95%的年轻人没有经历过副作用
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(0)

    # 大约95%的老年人确实经历过副作用
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(1)

train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
train_labels,train_samples = shuffle(train_labels,train_samples)
print(train_samples.shape)

# 通过将每个特征缩放到给定的范围来转换特征。
scaler = MinMaxScaler(feature_range=(0,1))

scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))
# 我们只是根据fit_transform（）函数默认情况下不接受一维数据的情况，将数据重塑为2D。
print(train_samples.reshape(-1,1).shape)

(2100,)
(2100, 1)


In [ ]:
# 创建并且编译模型
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

model = Sequential([
    Dense(units=16, input_shape=(1,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=2, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# 训练模型
model.fit(
    x = scaled_train_samples,
    y = train_labels,
    validation_split=0.1,
    batch_size = 10,
    epochs = 30,
    verbose=2
)

Epoch 1/30
189/189 - 1s - loss: 0.6783 - accuracy: 0.5968 - val_loss: 0.6701 - val_accuracy: 0.5810
Epoch 2/30
189/189 - 0s - loss: 0.6506 - accuracy: 0.6619 - val_loss: 0.6476 - val_accuracy: 0.6143
Epoch 3/30
189/189 - 0s - loss: 0.6234 - accuracy: 0.7201 - val_loss: 0.6240 - val_accuracy: 0.6524
Epoch 4/30
189/189 - 0s - loss: 0.5965 - accuracy: 0.7492 - val_loss: 0.5999 - val_accuracy: 0.6952
Epoch 5/30
189/189 - 0s - loss: 0.5673 - accuracy: 0.7820 - val_loss: 0.5713 - val_accuracy: 0.7476
Epoch 6/30
189/189 - 0s - loss: 0.5372 - accuracy: 0.8122 - val_loss: 0.5449 - val_accuracy: 0.7524
Epoch 7/30
189/189 - 0s - loss: 0.5082 - accuracy: 0.8259 - val_loss: 0.5172 - val_accuracy: 0.7810
Epoch 8/30
189/189 - 0s - loss: 0.4801 - accuracy: 0.8460 - val_loss: 0.4907 - val_accuracy: 0.8095
Epoch 9/30
189/189 - 0s - loss: 0.4539 - accuracy: 0.8587 - val_loss: 0.4666 - val_accuracy: 0.8286
Epoch 10/30
189/189 - 0s - loss: 0.4296 - accuracy: 0.8683 - val_loss: 0.4425 - val_accuracy: 0.8667

现在，我们不仅可以看到我们的模型对训练数据的特征的学习程度如何，还可以看到模型对验证集中的新的，看不见的数据的概括程度。 接下来，我们将看到如何使用我们的模型进行预测。